# SDK v1 code

In [ ]:
import pandas as pd
import numpy as np
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import AmlCompute,ComputeTarget, ComputeInstance
from azureml.exceptions import ComputeTargetException
from azureml.core.datastore import Datastore
from azureml.widgets import RunDetails
from azureml.core.environment import CondaDependencies

In [ ]:
ws = Workspace.from_config(path='../../config/config.json')

## Environment

In [ ]:
env=Environment.from_pip_requirements("conv_sum",  '../config/requirements.txt')
env.register(ws)

## Compute

In [ ]:
from ComputeManagement import create_cluster, create_instance, delete_compute

In [ ]:
cc=create_cluster(
    workspaceRef=ws,
    name="q34",
    vmSize="Standard_DS3_v2",
    minNodes=0,
    maxNodes=4,
    idleTime=180
)

## Training

In [ ]:
# training_script_config = ScriptRunConfig(
#     source_directory = 'src',
#     script = 'training_script.py',
#     arguments=['--data',___],
#     environment = env,
#     compute_target = cc
# )
# experiment = Experiment(
#     workspace = ws,
#     name="maiden_experiment"
# )
# run = experiment.submit(config=training_script_config, tags=[])

# RunDetails(run).show()
# run.wait_for_completion(show_output=True)

In [ ]:
import yfinance as yf

In [ ]:
sensexTickerYFinance = ['HDFCLIFE.NS, NESTLEIND.NS, KOTAKBANK.NS, INDUSINDBK.NS, TATASTEEL.NS, ITC.NS, ONGC.NS, TITAN.NS, ULTRACEMCO.NS, BAJAJFINSV.NS, BAJFINANCE.NS, BRITANNIA.NS, BAJAJ-AUTO.NS, COALINDIA.NS, BHARTIARTL.NS, TATACONSUM.NS, LTI.NS, CIPLA.NS, MARUTI.NS, ICICIBANK.NS, APOLLOHOSP.NS, NTPC.NS, HEROMOTOCO.NS, HINDALCO.NS, WIPRO.NS, TCS.NS, ADANIENT.NS, MM.NS, TECHM.NS, RELIANCE.NS']
stock_data = yf.download(tickers=sensexTickerYFinance, start='2000-01-01', end='2022-12-31', interval='1mo')

In [ ]:
stock_data.loc[:,'Adj Close']

 TODO
1. Read about SOTA for stock price prediction and what determines success of model trying to predict price
2. Choose stocks to monitor - Nifty 50
3. Build as below

Think of a common use-case where data would update regularly and model would shift
1. Stock price prediction
2. Automatic data retrieval using API to store into Azure storage
3. Automatic model training at intervals depending on error rate

Tie everything up in a RL portfolio optimization application

In [ ]:
import yfinance as yf

In [ ]:
tickerData= yf.download(tickers="RELIANCE.NS", start="2022-01-01", end="2023-01-10", period="1d")
tickerData['Date'] = [str(x)[:10] for x in tickerData.index]
tickerData['Ticker'] = "RELIANCE.NS"
tickerDataToPersist = list(tickerData.transpose().to_dict().values())

In [ ]:
from src.TickerData import query, download

In [ ]:
download(ticker="RELIANCE.NS", start="2022-12-01",end="2023-01-10", period="1d")

In [ ]:
train_data.to_csv()

In [ ]:
train_data = query(ticker="RELIANCE.NS", start="2022-12-01",end="2023-01-10")

In [ ]:
train_data

In [ ]:
import pandas as pd

In [ ]:
import torch

In [ ]:
ticker="RELIANCE.NS"

In [ ]:
train_data = train_data[f"{ticker}_Close"]

In [ ]:
import pandas as pd

In [ ]:
pd.Series(train_data).to_csv('./data/ril.csv')

In [ ]:
import numpy as np

In [ ]:
def training_data(series, loookaheadSize=5):
    X,y = [],[]
    for i in np.arange(5,len(series)-1):
        X.append(series[i-loookaheadSize:i])
        y.append(series[i+1])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(len(series)-loookaheadSize-1,1,5)
    y=y.reshape(-1,1)

    train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X), torch.from_numpy(y))

    return train_dataset

In [ ]:
tx=training_data(train_data)

In [ ]:
next(iter(tx))

In [ ]:
torch.save(tx,'txx.pt')

In [ ]:
typ = torch.load('txx.pt')

In [ ]:
next(iter(typ))

In [ ]:
next(iter(tx))

In [ ]:
import numpy as np

In [ ]:
np.array([0.0026]).shape

In [ ]:
from datetime import datetime
str(datetime.now().date())

# SDK v2 code

In [ ]:
import json
import numpy as np
from azure.ai.ml import MLClient, Input, Output, command
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import AmlCompute, Environment, Model, Data, CodeConfiguration, ManagedOnlineEndpoint, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes
from datetime import datetime

## Init

In [ ]:
with open('../config/config.json','r') as f:
    configs=json.loads(f.read())

subscription_id, resource_group, workspace = configs['subscription_id'], configs['resource_group'], configs['workspace_name']

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ml_client = MLClient(
    credential, subscription_id, resource_group, workspace
)

## Upload data to Azure

In [ ]:
my_path = '../data/ril.csv'



In [ ]:
ril_data = ml_client.data.get(name='ril', version=1)

In [ ]:
ril_data.path

## Training

## Run training job

In [ ]:
def version_iter(n=20):
    i=0
    for i in np.arange(21,50):
        yield i
x = iter(version_iter())
next(x)

In [ ]:
local_model_name = f"modelstock_pred_{str(datetime.now().date())}"
registered_model_name = "stock_pred_v1"

job = command(   
    name=f"stock_pred_job_{next(x)}",
    inputs={
        "data": Input(type=AssetTypes.URI_FILE, mode="ro_mount", path=ril_data.path),
        "test_train_ratio": 0.25,
        "registered_model_name":registered_model_name,
        "local_model_name":local_model_name
        },
    code="../src/",  # location of source code
    command="python train.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --local_model_name ${{inputs.local_model_name}} --registered_model_name ${{inputs.registered_model_name}}",
    environment=env,
    compute=compute.name,
    experiment_name="train_model_stock_price_prediction",
    display_name="stock_price_prediction",
)

ml_client.create_or_update(job)

## Register the trained model

In [ ]:
local_model_name=f"modelstock_pred_{str(datetime.now().date())}"

In [ ]:
local_model_name

In [ ]:
model = Model(
        path=f"azureml://jobs/{job.name}/outputs/artifacts/paths/outputs/",
        name="model-path",
        description="Model created from run.",
        type="custom_model",
    )

In [ ]:
ml_client.models.create_or_update(model)

## Deployment

In [ ]:
# Create endpoint
online_endpoint_name = "modeldir-logged-init"

endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint"
)

ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state

In [ ]:
## Create deployment only after endpoint has provisioned

In [ ]:
# Deployment script
code_config = CodeConfiguration(
        code=src_dir, scoring_script="deployment.py"
    )

# Create deployment
yellow_deployment = ManagedOnlineDeployment(
    name="yellow",
    endpoint_name=online_endpoint_name,
    model=modeldir,
    environment=Environment(
            conda_file=f"./{dependencies_dir}/conda.yml",
            image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04"),
    code_configuration=code_config,
    instance_type="Standard_DS2_v2",
    instance_count=1,
)
# create the deployment:
ml_client.begin_create_or_update(yellow_deployment)
# blue deployment takes 100 traffic
# endpoint.traffic = {"yellow": 100}
# ml_client.begin_create_or_update(endpoint)

## Test the endpoint

In [ ]:
%%writefile inputs.json
{"d1":2663,"d2":2654.4,"d3":2698,"d4":2690,"d5":2698.12}

In [ ]:
# test the endpoint (the request will route to blue deployment as set above)
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="yellow",
    request_file="inputs.json",
)

In [ ]:
ml_client.compute.begin_delete(name=compute.name).wait()

## Deployed endpoint logs

In [ ]:
ml_client.online_deployments.get_logs(
    name="yellow", endpoint_name=online_endpoint_name, lines=50
)